In case it´s necessary unccomment the next line to install HuggingFace Transformers and Datasets library

In [ ]:
#! pip install datasets transformers seqeval

For fine-tuning the models it´s necessary to authenticate you´r HuggingFace user. Execute the next cell and write your user´s token

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
#!apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.19.2


# Fine-tuning a model on a token classification task

The task we are working on is token classification. It consist on classifing each token in a document. For that we have to have the corresponding labels in a form of a list together with the tokenized text. Esssentially we are working with a NER task at token level.

We load the corresponding model.

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"

## Loading the dataset

We will use the Datasets library from Huggingface. This library has some interesting methos as load_dataset for loading the dataset from many file resoruces and load_metric which alows us to calculate the metrics automatically.

In [ ]:
from datasets import load_dataset, load_metric, Features, Value, ClassLabel, Dataset, DatasetDict

We define the two classes for our tokens. In this case we have O for the tokens which are no event and B-MISC for the tokens that are events.

As the CSV we have is built with lists of tags and strings we can´t load the classlabels directly with the load_dataset method. Even thought we can define the features for future use.

In [ ]:
label_list = ['O', 'B-MISC']
event_features = Features({'string': Value('string'), 'begin': Value('string'), 'end': Value('string'), 'tag': ClassLabel(num_classes=2, names=class_names)})
dataset_csv = load_dataset("csv", data_files={'train': 'events_train.csv', 'test': 'events_test.csv', 'validation': 'events_dev.csv'})


We can check the dataset structure with the next line

In [ ]:
dataset_csv

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset_csv["train"])

,file,string,begin,end,tag
0,ES100737.xml,"['Paciente', 'de', '5', 'años', 'y', '7', 'meses', 'de', 'edad', ',', 'sin', 'antecedentes', 'personales', 'o', 'familiares', 'de', 'interés', ',', 'ingresó', 'en', 'nuestro', 'hospital', 'el', '12', 'de', 'junio', 'de', '2002', 'por', 'presentar', 'un', 'cuadro', 'de', '10', 'días', 'de', 'evolución', 'consistente', 'en', 'cefalea', 'de', 'predominio', 'vespertino', ',', 'al', 'que', 'se', 'añade', 'posteriormente', 'estrabismo', 'convergente', 'del', 'ojo', 'izquierdo', 'y', 'diplopia', '.', 'En', 'la', 'exploración', 'neurológica', 'nos', 'encontramos', 'con', 'una', 'niña', 'despierta', ',', 'colaboradora', 'y', 'conectada', 'ambientalmente', ',', 'que', 'presentaba', 'estrabismo', 'convergente', 'de', 'ojo', 'izquierdo', 'con', 'motilidad', 'ocular', 'conservada', 'y', 'nistagmo', 'horizontal', 'a', 'la', 'mirada', 'forzada', 'hacia', 'la', 'derecha', '.', 'La', 'analítica', 'de', 'sangre', ',', 'bioquímica', 'y', 'radiografía', 'de', 'tórax', 'resultaron', 'normales', '.', 'La', 'TAC', 'y', 'RM', 'de', 'cráneo', 'mostraron', 'una', 'masa', 'bien', 'delimitada', 'localizada', 'en', 'el', 'interior', 'del', 'ventrículo', 'lateral', 'derecho', 'a', 'nivel', 'del', 'atrio', 'y', 'asta', 'temporal', ',', 'de', 'densidad', 'homogénea', 'y', 'de', '6', 'por', '2,5', 'cms', ',', 'con', 'importante', 'edema', 'perilesional', 'y', 'que', 'mostraba', 'intensa', 'captación', 'tras', 'la', 'administración', 'de', 'contraste', 'intravenoso', '.', 'La', 'paciente', 'fue', 'intervenida', 'mediante', 'abordaje', 'trans-temporal', 'a', 'través', 'de', 'la', 'circunvolución', 'temporal', 'media', 'con', 'extirpación', 'macroscópicamente', 'completa', 'de', 'la', 'tumoración', '.', 'El', 'estudio', 'histopatológico', 'evidenció', 'una', 'proliferación', 'neoplásica', 'meningotelial', 'con', 'patrón', 'predominantemente', 'meningoteliomatoso', 'y', 'elevada', 'celularidad', ',', 'tipificándose', 'el', 'tumor', 'como', 'meningioma', 'atípico', '.', 'Llamaba', 'la', 'atención', 'la', 'presencia', 'de', 'una', 'elevada', 'actividad', 'mitótica', 'que', ',', 'en', 'alguna', 'zona', ',', 'llegaba', 'a', 'ser', 'de', '2', '-', '3', '-', 'mitosis', 'en', '10', 'campos', 'de', 'gran', 'aumento', ',', 'presentando', 'las', 'células', 'proliferantes', 'núcleos', 'más', 'voluminosos', 'y', 'nucleolo', 'prominente', '.', 'La', 'actividad', 'del', 'anticuerpo', 'Ki', '-', '67', ',', 'medida', 'de', 'forma', 'aleatoria', 'en', 'la', 'totalidad', 'del', 'tumor', ',', 'oscilaba', 'en', 'torno', 'al', '18', '-', '20%', '.', 'La', 'evolución', 'clínica', 'de', 'la', 'niña', 'fue', 'favorable', ',', 'siendo', 'dada', 'de', 'alta', 'hospitalaria', 'a', 'los', 'diez', 'días', 'de', 'la', 'intervención', 'quirúrgica', 'con', 'un', 'mínimo', 'estrabismo', 'convergente', 'de', 'ojo', 'derecho', '.', 'A', 'pesar', 'de', 'no', 'haber', 'sido', 'sometida', 'a', 'tratamientos', 'complementarios', ',', 'en', 'la', 'actualidad', ',', 'la', 'paciente', 'se', 'encuentra', 'asintomática', 'y', 'no', 'se', 'han', 'evidenciado', 'signos', 'de', 'recidiva', 'en', 'los', 'estudios', 'de', 'imagen', 'de', 'control', '.']","['0', '9', '12', '14', '19', '21', '23', '29', '32', '36', '38', '42', '55', '66', '68', '79', '82', '89', '91', '99', '102', '110', '119', '122', '125', '128', '134', '137', '142', '146', '156', '159', '166', '169', '172', '177', '180', '190', '202', '205', '213', '216', '227', '237', '239', '242', '246', '249', '255', '270', '281', '293', '297', '301', '311', '313', '321', '323', '326', '329', '341', '353', '357', '369', '373', '377', '382', '391', '393', '406', '408', '418', '432', '434', '438', '449', '460', '472', '475', '479', '489', '493', '503', '510', '521', '523', '532', '543', '545', '548', '555', '563', '569', '572', '579', '582', '585', '595', '598', '604', '606', '617', '619', '631', '634', '640', '651', '659', '661', '664', '668', '670', '673', '676', '683', '693', '697', '702', '707', '718', '729', '73

## Preprocessing the data

We use the AutoTokenizer for loading the corresponding tokenizer for our model. 

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizers. Those fast tokenizers are available for almost all models

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

The labels of all special tokenks introduced by the tokenizer are set to -100 (the index that is ignored by PyTorch) and the resto of the tokens are set to the word id they come from.

The next cell defines the way we are going to tokenize the dataset. We feed the rows to the tokenizer with truncation=True so we can process bigger text that the model can´t handle by itself. Also we set the argument is_split_into_words=True so the text is split into words. After that, the labels and the tokens id´s are alligned

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["string"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["tag"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    #tokenized_inputs["labels"] = labels
    return tokenized_inputs

For simplifying the process of tokenization we use de map method together with the method we have just define, so the three datasets can be processed within one line

In [ ]:
from datasets import Sequence

tokenized_dataset = dataset.map(tokenize_and_align_labels_, batched=True)



We can check the dataset features with the next cell

In [ ]:
print(tokenized_dataset['train'].features)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

As we have done with the tokenizer we download the model with the AutoModelForTokenClassification which prepares the model for the task. The method removes the last layer of the model and adds another one that´s more suitable for the token classification. We have to define the number of labels for the classification. We define label_list again in case the cell it´s defined has not been executed.

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_list = ['O', 'B-MISC']

model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(class_names))

We define the arguments for the Trainer, including the batch size, number of epochs and weight decay. For uploading the model to the HuggingFace hub just uncomment the push_to_hub line.

In [ ]:
model_name = model_checkpoint.split("/")[-1]
batch_size_ = 4
lr = 2e-5
wd = 0.01
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=lr,
    per_device_train_batch_size=batch_size_,
    per_device_eval_batch_size=batch_size_,
    num_train_epochs=24,
    weight_decay=wd,
    #push_to_hub=True,
    logging_steps = 1
)

Usually the model includes a DataCollator, but to ensure it process the data correctly we define one. The data collator is in charge of applyinh padding to make all the inputs the same size.

In [ ]:
from transformers import DataCollatorForTokenClassification
label_list = ['O','B-MISC']

data_collator = DataCollatorForTokenClassification(tokenizer)

For calculating the metrics we use the seqeval metric defined in HuggingFace library (https://github.com/chakki-works/seqeval).

In [ ]:
metric = load_metric("seqeval")

We create the csv for storing the results, just uncomment the .to_csv() line.

In [ ]:
columns = ['run', 'precision', 'recall', 'f1', 'accuracy', 'learning_rate', 'weight_decay', 'batch_size', 'training_loss', 'evaluation_loss']
results_df = pd.DataFrame(columns = columns)
#results_df.to_csv('resultados_train.csv', header = True)

We calculate the false positives and negatives for further studies but they are not necessary for calculating the precision, recall and F-1 because it´s done by the metric defined. We store the results in a csv, in case you want to store the results just uncomment the line .to_csv(). We store multiple metrics, as the loss computed the batch size and weight decay defined and all the metrics calculated. 

We calculate the false positives and negatives for further studies but they are not necessary for calculating the precision, recall and F-1 because it´s done by the metric defined. We store the results in a csv, in case you want to store the results just uncomment the line 

In [ ]:
import numpy as np
import pandas as pd
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_predictions_final = []
    true_labels_final = [] 
    tp = 0
    fn = 0
    fp = 0
    for x, y in zip(true_predictions, true_labels):
      true_predictions_aux = x
      true_labels_aux = y
      for i, j in zip(x, y):
        if i == 'B-MISC' and i == j:
            tp+=1
            true_predictions_aux.pop(0)
            true_labels_aux.pop(0)
        elif i == 'O' and i == j:
            true_predictions_aux.pop(0)
            true_labels_aux.pop(0)
      true_predictions_final.append(true_predictions_aux)
      true_labels_final.append(true_labels_aux)
    
    for l, ll in zip(true_labels_aux, true_predictions_aux):
      fn += len(l)
      fp += len(ll)

    results = metric.compute(predictions=true_predictions, references=true_labels)
    
    results_list = [[2, results["overall_precision"], results["overall_recall"], results["overall_f1"], results["overall_accuracy"], lr, wd, batch_size_, 0, 0]]
    columns = ['run', 'precision', 'recall', 'f1', 'accuracy', 'learning_rate', 'weight_decay', 'batch_size', 'training_loss', 'evaluation_loss']
    results_df = pd.DataFrame(results_list, columns = columns)

    #results_df.to_csv('resultados_train.csv', mode='a', index = True, header = False)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

We create the Trainer object with all we have defined

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We train the model 

In [ ]:
train_result = trainer.train()

The messages of the columns that are not being used by the model is completely normal. The model only use the columns we created in the tokenizing phase.

With the next cell we can check the training results

In [ ]:
print(train_result)

TrainOutput(global_step=48, training_loss=0.23017056426033378, metrics={'train_runtime': 42.6169, 'train_samples_per_second': 9.011, 'train_steps_per_second': 1.126, 'total_flos': 100337954586624.0, 'train_loss': 0.23017056426033378, 'epoch': 16.0})


The evaluate method allows us to check which model has been the best in the training process and the results of it on the evaluation process

In [ ]:
trainer.evaluate()

We can calculate the final results on the test dataset. We calculate the results on the MISC tag, since it determines how good the events are detected. 

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: file, string, end, __index_level_0__, begin, tag. If file, string, end, __index_level_0__, begin, tag are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 45
  Batch size = 4


{'MISC': {'f1': 0.869385505648939,
  'number': 3519,
  'precision': 0.843808504947847,
  'recall': 0.8965615231599886},
 'overall_accuracy': 0.9524406762654893,
 'overall_f1': 0.869385505648939,
 'overall_precision': 0.843808504947847,
 'overall_recall': 0.8965615231599886}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
#trainer.push_to_hub()